Can you help detect toxic comments ― and minimize unintended model bias? That's your challenge in this competition.

The Conversation AI team, a research initiative founded by Jigsaw and Google (both part of Alphabet), builds technology to protect voices in conversation. A main area of focus is machine learning models that can identify toxicity in online conversations, where toxicity is defined as anything rude, disrespectful or otherwise likely to make someone leave a discussion.

Last year, in the Toxic Comment Classification Challenge, you built multi-headed models to recognize toxicity and several subtypes of toxicity. This year's competition is a related challenge: building toxicity models that operate fairly across a diverse range of conversations.

Here’s the background: When the Conversation AI team first built toxicity models, they found that the models incorrectly learned to associate the names of frequently attacked identities with toxicity. Models predicted a high likelihood of toxicity for comments containing those identities (e.g. "gay"), even when those comments were not actually toxic (such as "I am a gay woman"). This happens because training data was pulled from available sources where unfortunately, certain identities are overwhelmingly referred to in offensive ways. Training a model from data with these imbalances risks simply mirroring those biases back to users.

In this competition, you're challenged to build a model that recognizes toxicity and minimizes this type of unintended bias with respect to mentions of identities. You'll be using a dataset labeled for identity mentions and optimizing a metric designed to measure unintended bias. Develop strategies to reduce unintended bias in machine learning models, and you'll help the Conversation AI team, and the entire industry, build models that work well for a wide range of conversations.

Disclaimer: The dataset for this competition contains text that may be considered profane, vulgar, or offensive.

Acknowledgments
The Conversation AI team would like to thank Civil Comments for making this dataset available publicly and the Online Hate Index Research Project at D-Lab, University of California, Berkeley, whose labeling survey/instrument informed the dataset labeling. We'd also like to thank everyone who has contributed to Conversation AI's research, especially those who took part in our last competition, the success of which led to the creation of this challenge.

This is a Kernels-only competition. Refer to Kernels Requirements for details.


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler # Used for scaling of data
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt


Using TensorFlow backend.


In [0]:
#!pip install kaggle --upgrade

In [0]:
#!pip install -U -q kaggle
#!mkdir -p ~/.kaggle

In [0]:
!pip install -U -q kaggle

In [0]:
from google.colab import files
files.upload()

MessageError: ignored

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets list

ref                                                     title                                              size  lastUpdated          downloadCount  
------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
himanshupoddar/zomato-bangalore-restaurants             Zomato Bangalore Restaurants                       88MB  2019-03-31 09:48:43           4383  
thegurus/spanish-high-speed-rail-system-ticket-pricing  Spanish High Speed Rail tickets pricing - Renfe    27MB  2019-05-10 06:50:30           1119  
crisparada/brazilian-cities                             Brazilian Cities                                  935KB  2019-05-24 20:49:21           1777  
chicago/chicago-copa-cases                              Chicago COPA Cases                                  3MB  2019-04-05 00:00:32           1765  
taniaj/australian-election-2019-tweets                  Australian Election 2019 Tweets             

In [0]:

!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 6, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
!ls

kaggle.json  sample_submission.csv.zip	train.csv.zip
sample_data  test.csv.zip


In [0]:













import pandas as pd
#submisson = pd.read_csv('/content/sample_submission.csv.zip', compression='zip', header=0, sep=',', quotechar='"')


In [0]:
import pandas as pd
#train = pd.read_csv('/content/train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')


In [0]:
import pandas as pd
#test = pd.read_csv('/content/test.csv.zip', compression='zip', header=0, sep=',', quotechar='"')


In [0]:
#test.head()

In [0]:
#train.head()

In [0]:
#type(test)

In [0]:
#type(train)

In [0]:
#data_train.info()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:

from zipfile import ZipFile
zip_file = ZipFile('/content/test.csv.zip')
data_test=pd.read_csv(zip_file.open('test.csv'))
data_test.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


In [0]:
from zipfile import ZipFile
zip_file = ZipFile('/content/sample_submission.csv.zip')
data_submission=pd.read_csv(zip_file.open('sample_submission.csv'))
data_submission.head()

,id,prediction
0,7000000,0.0
1,7000001,0.0
2,7000002,0.0
3,7000003,0.0
4,7000004,0.0


In [0]:
#}#dataset.head(2)

In [0]:

#corpus = data_train.loc[:, ["comment_text"]]


In [0]:
#type(corpus)
#data_train["comment_text"]

In [0]:
#corpus1=corpus.values.tolist()

In [0]:
#type(corpus1)

In [0]:
#import itertools


In [0]:
#corpus1=list(itertools.chain(*corpus1))

In [0]:


#from sklearn.feature_extraction import text
#from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
import re
data_train["comment_text"]=[re.sub('[0-9!`@#$%^&*"{"|"}"<>.?/*+-_;:]+', '', _) for _ in data_train["comment_text"]]

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords



data_train["comment_text"]=data_train["comment_text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))



data_train["comment_text"]=data_train["comment_text"].str.replace('\d+', '')

data_train["comment_text"]=data_train["comment_text"].str.replace('\([^()]*\)', '')

data_train["comment_text"]=data_train["comment_text"].str.replace('[!@#$%^&*(),.?":{}|<>]', '')

data_train["comment_text"]=data_train["comment_text"].str.replace('[+]', '')

data_train["comment_text"]=data_train["comment_text"].str.replace('[-]', '')


data_train["comment_text"] = data_train["comment_text"].str.replace('[^\w\s]','')


from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


tfidf=TfidfVectorizer(min_df=2,max_df=0.5,ngram_range=(1,2))


features=tfidf.fit(data_train["comment_text"])

measures=features.transform(data_train["comment_text"])


df=pd.DataFrame(
        measures.todense(),
        columns=tfidf.get_feature_names()
        )



In [0]:

my_stop_words = text.ENGLISH_STOP_WORDS.union(["book"])

vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)

X = vectorizer.fit_transform(my_list)

idf_values = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))


In [0]:
X


<1804874x573457 sparse matrix of type '<class 'numpy.float64'>'
	with 39256878 stored elements in Compressed Sparse Row format>

In [0]:

# printing the tfidf vectors
print(X)

# printing the vocabulary
print(vectorizer.vocabulary_)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
#import tensorflow-gpu

SyntaxError: ignored

In [0]:
"""feature_names = vectorizer.get_feature_names()
for col in X.nonzero()[1]:
    print (feature_names[col], ' - ', X[0, col])"""


"feature_names = vectorizer.get_feature_names()\nfor col in X.nonzero()[1]:\n    print (feature_names[col], ' - ', X[0, col])"

In [0]:
data_train.severe_toxicity=pd.to_numeric(data_train.severe_toxicity, errors='ignore')

In [0]:
data_train[data_train.severe_toxicity >= 0.5].severe_toxicity = 1
data_train[data_train.severe_toxicity < 0.5].severe_toxicity = 0

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
data_train.severe_toxicity.describe()

count    1.804874e+06
mean     4.582099e-03
std      2.286128e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: severe_toxicity, dtype: float64

In [0]:
data_train.severe_toxicity

0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.021277
5          0.047619
6          0.050847
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13         0.000000
14         0.000000
15         0.000000
16         0.000000
17         0.000000
18         0.000000
19         0.000000
20         0.000000
21         0.000000
22         0.000000
23         0.000000
24         0.000000
25         0.000000
26         0.000000
27         0.000000
28         0.000000
29         0.000000
             ...   
1804844    0.000000
1804845    0.000000
1804846    0.000000
1804847    0.000000
1804848    0.000000
1804849    0.000000
1804850    0.000000
1804851    0.000000
1804852    0.000000
1804853    0.000000
1804854    0.000000
1804855    0.000000
1804856    0.000000
1804857    0.000000
1804858    0.000000
1804859    0.000000
1804860    0.000000
1804861    0.000000
1804862    0.000000


In [0]:
#from sklearn.linear_model import LinearRegression


In [0]:
#moddel=LinearRegression().fit(X, data_train.severe_toxicity)

In [0]:
#out=moddel.predict(X)

In [0]:
#out

array([ 2.78370033e-03,  1.65483225e-04,  3.90848637e-03, ...,
        1.00110655e-03,  1.91258748e-02, -5.23546518e-08])

In [0]:
#len(out)

1804874

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = data_test.loc[:, ["comment_text"]]

In [0]:

tfidf1=TfidfVectorizer(min_df=2,max_df=0.5,ngram_range=(1,2))


features1=tfidf1.fit(data['vendor'])

measures1=features1.transform(data['vendor'])


df1=pd.DataFrame(
        measures1.todense(),
        columns=tfidf1.get_feature_names()
        )


df1.head()

df1.columns

In [0]:
type(corpus)
data_test.loc[:, ["comment_text"]]

,comment_text
0,Jeff Sessions is another one of Trump's Orwell...
1,I actually inspected the infrastructure on Gra...
2,No it won't . That's just wishful thinking on ...
3,Instead of wringing our hands and nibbling the...
4,how many of you commenters have garbage piled ...
5,Why can't the Globe & Mail provide the symbols...
6,"That's already been happening, Carl, it's call..."
7,"imagine the costs for security, transportation..."
8,So they want to put United on the board of thi...
9,"No, no that can't be. It's Russia that's the ..."


In [0]:
corpus1=corpus.values.tolist()

In [0]:

import itertools
corpus1=list(itertools.chain(*corpus1))
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
import re
my_list= corpus1
my_list=[re.sub('[0-9!`@#$%^&*"{"|"}"<>.?/*+-_;:]+', '', _) for _ in my_list]
my_stop_words = text.ENGLISH_STOP_WORDS.union(["book"])

vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)

X1 = vectorizer.fit_transform(my_list)

idf_values = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))


In [0]:
idf_values

{'aa': 9.441247634501433,
 'aaa': 10.406328530545021,
 'aaaa': 10.694010602996801,
 'aaaaa': 11.387157783556747,
 'aaaaaaa': 11.79262289166491,
 'aaaaaaaaaaaaa': 11.79262289166491,
 'aaaaaaaaaaaaaaa': 11.79262289166491,
 'aaaaaaaaaaaaaaaaa': 11.79262289166491,
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaa': 11.79262289166491,
 'aaaaaaaaayou': 11.79262289166491,
 'aaaaaaaar': 11.79262289166491,
 'aaaaaaand': 11.79262289166491,
 'aaaaalmost': 11.79262289166491,
 'aaaaap': 11.79262289166491,
 'aaaaargh': 11.79262289166491,
 'aaaaay': 11.79262289166491,
 'aaaah': 11.099475711104967,
 'aaaaiit': 11.79262289166491,
 'aaaand': 11.79262289166491,
 'aaaay': 11.79262289166491,
 'aaack': 11.79262289166491,
 'aaargh': 11.79262289166491,
 'aaathat': 11.79262289166491,
 'aaayeaaah': 11.79262289166491,
 'aaazing': 11.79262289166491,
 'aab': 11.099475711104967,
 'aactually': 11.79262289166491,
 'aad': 10.876332159790756,
 'aada': 11.79262289166491,
 'aadereinhoff': 11.79262289166491,
 'aafala': 10.406328530545021,
 

In [0]:
from keras import Sequential
from keras.layers import Dense

In [0]:
import pandas as pd

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [0]:
#X=pd.DataFrame(X.todense())
#X1=pd.DataFrame(X1.todense())
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
X.shape


(1804874, 573457)

In [0]:
X1.shape

(97320, 97687)

In [0]:
#type(X)
#type(X1)
#import numpy as np
#val=np.random.choice(X, 3, replace=False)

scipy.sparse.csr.csr_matrix

In [0]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal', input_dim=X.shape[1]))
#Second  Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [0]:
#Fitting the data to the training dataset
classifier.fit(X,data_train.severe_toxicity, batch_size=9, epochs=1)

Epoch 1/1
 514269/1804874 [=======>......................] - ETA: 5:18:53 - loss: 0.0285 - acc: 0.9423Buffered data was truncated after reaching the output size limit.

In [0]:
import pickle
path= "mymodel_Dl.pkl"
with open(path, 'wb') as f:
        pickle.dump(model, f)
        print("Done Pickiling")

In [0]:
eval_model=classifier.evaluate(X, data_train.severe_toxicity)
eval_model

In [0]:
y_pred=classifier.predict(X1)


In [0]:
max(y_pred)
min(y_pred)

In [0]:
y_pred =(y_pred>0.5)

In [0]:
eval_model=classifier.evaluate(X, data_train.severe_toxicity)
eval_model
y_pred=classifier.predict(X1)

max(y_pred)
min(y_pred)
y_pred =(y_pred>0.5)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)